In [1]:
import io
import os
import sys
import json
import pandas as pd
import numpy as np
import regex as re

# Pets Dataframe

In [3]:
with io.open(os.path.join(sys.path[0], "18.0_pets.json"), mode="r", encoding="utf-8") as f:
    data = json.load(f)
    
df = pd.DataFrame(data).explode('Abilities')
df.head()

,Attack,Health,Abilities,Id,Name,Tier,Packs,Rollable
0,2,1,"{'Level': 1, 'About': 'Faint: Give a random fr...",0,Ant,1,"[Pack1, Pack2]",True
0,2,1,"{'Level': 2, 'About': 'Faint: Give a random fr...",0,Ant,1,"[Pack1, Pack2]",True
0,2,1,"{'Level': 3, 'About': 'Faint: Give a random fr...",0,Ant,1,"[Pack1, Pack2]",True
1,3,2,"{'Level': 1, 'About': 'Sell: Give 2 random fri...",3,Beaver,1,"[Pack1, Pack2]",True
1,3,2,"{'Level': 2, 'About': 'Sell: Give 2 random fri...",3,Beaver,1,"[Pack1, Pack2]",True


In [4]:
# cleaning abilities
abilities = df.Abilities.apply(pd.Series)
df1 = pd.concat([df.drop('Abilities', axis = 1), abilities], axis = 1)

# custom abilities
df1.loc[df1.Name == 'Sloth', 'About'] = 'No ability.'
df1.loc[df1.Name == 'Cat', 'About'] = 'Aura: ' + df1.loc[df1.Name == 'Cat', 'About']
df1.loc[df1.Name == 'Tiger', 'About'] = 'Aura: ' + df1.loc[df1.Name == 'Tiger', 'About']

def add_no_ability(about):
    if about == 'No ability.':
        return 'None: No ability.'
    else:
        return about

df1.About = df1.About.apply(add_no_ability)
df1.head()

,Attack,Health,Id,Name,Tier,Packs,Rollable,Level,About
0,2,1,0,Ant,1,"[Pack1, Pack2]",True,1,Faint: Give a random friend +2/+1.
0,2,1,0,Ant,1,"[Pack1, Pack2]",True,2,Faint: Give a random friend +4/+2.
0,2,1,0,Ant,1,"[Pack1, Pack2]",True,3,Faint: Give a random friend +6/+3.
1,3,2,3,Beaver,1,"[Pack1, Pack2]",True,1,Sell: Give 2 random friends +1 Health.
1,3,2,3,Beaver,1,"[Pack1, Pack2]",True,2,Sell: Give 2 random friends +2 Health.


In [5]:
about = df1.About.str.split(':', expand=True)
about.columns = ['Trigger', 'Ability']
df2 = pd.concat([df1.drop('About', axis = 1), about], axis = 1)
df2.Ability = df2.Ability.str.lstrip()

def get_n_uses(ability):
    n_uses = 999
    if ability.endswith('trigger(s).'):
        n_uses = int(ability.split()[-2])
    return n_uses
df2['N_uses'] = df2.Ability.apply(get_n_uses)

df2.head()

,Attack,Health,Id,Name,Tier,Packs,Rollable,Level,Trigger,Ability,N_uses
0,2,1,0,Ant,1,"[Pack1, Pack2]",True,1,Faint,Give a random friend +2/+1.,999
0,2,1,0,Ant,1,"[Pack1, Pack2]",True,2,Faint,Give a random friend +4/+2.,999
0,2,1,0,Ant,1,"[Pack1, Pack2]",True,3,Faint,Give a random friend +6/+3.,999
1,3,2,3,Beaver,1,"[Pack1, Pack2]",True,1,Sell,Give 2 random friends +1 Health.,999
1,3,2,3,Beaver,1,"[Pack1, Pack2]",True,2,Sell,Give 2 random friends +2 Health.,999


In [6]:
# need to extract target_func and effect_func from Ability 
abls = df2.Ability
abls

0                            Give a random friend +2/+1.
0                            Give a random friend +4/+2.
0                            Give a random friend +6/+3.
1                       Give 2 random friends +1 Health.
1                       Give 2 random friends +2 Health.
                             ...                        
174                       Give 2 random friends Peanuts.
174                       Give 3 random friends Peanuts.
175    Make 1 friend behind level 3. Except other Whi...
175    Make 2 friends behind level 3. Except other Wh...
175    Make 3 friends behind level 3. Except other Wh...
Name: Ability, Length: 528, dtype: object

In [11]:
# cleaning packs
bool_dict = {}
for item in ['Pack1', 'Pack2', 'Pack3', 'Custom']:
    bool_dict[item] = df1.Packs.apply(lambda x: item in x)
packs = pd.DataFrame(bool_dict)

df3 = pd.concat([df2.drop('Packs', axis = 1), packs], axis = 1).set_index(['Name', 'Level'])
df3.head()

Attack  Health Id  Tier  Rollable Trigger  \
Name   Level                                              
Ant    1           2       1  0     1      True   Faint   
       2           2       1  0     1      True   Faint   
       3           2       1  0     1      True   Faint   
Beaver 1           3       2  3     1      True    Sell   
       2           3       2  3     1      True    Sell   

                                       Ability  N_uses  Pack1  Pack2  Pack3  \
Name   Level                                                                  
Ant    1           Give a random friend +2/+1.     999   True   True  False   
       2           Give a random friend +4/+2.     999   True   True  False   
       3           Give a random friend +6/+3.     999   True   True  False   
Beaver 1      Give 2 random friends +1 Health.     999   True   True  False   
       2      Give 2 random friends +2 Health.     999   True   True  False   

              Custom  
Name   Level          
Ant    1       False  
       2       False  
       3       False  
Beaver 1       False  
       2       False

*insert a quick export into Excel and manual tidying*

In [22]:
df4 = pd.read_excel (r'./pets_df.xlsx', index_col=[0,1])
df4.N_targets = df4.N_targets.astype(float)

def to_list(datum):
    if datum != datum:
        return np.nan
    if isinstance(datum, int):
        return [datum]
    if isinstance(datum, str):
        return eval(datum)

# df4.Target_pos = df4.Target_pos.apply(to_list)
# df4.head()

In [20]:
def get_effect(abl, target):
    if '+' in abl:
        return 'ModifyStats'
    if any([status in abl for status in ['Coconut', 'Melon', 'Peanut', 'Weak', 'Honey']]):
        return 'ApplyStatus'
    if 'damage' in abl:
        return 'DealDamage'
    if 'ummon' in abl:
        return 'Summon'
    if abl == 'No ability.':
        return 'None'
    if any([word in abl for word in ['Push', 'push']]):
        return 'Push'
    if any([word in abl for word in ['Swallow', 'swallow']]):
        return 'Swallow'
    return np.nan
    
# df4.Effect = df4.apply(lambda x: get_effect(x.Ability, x.Target), axis=1)

In [129]:
df4 = pd.read_excel (r'./pets_df.xlsx', index_col=[0,1])
df4.Ability = df4.Ability.str.replace('<b>','').str.replace('</b>', '')

def get_Effect_info(abl, effect, lvl, effect_info):
    if effect == 'ApplyStatus':
        for status in ['Coconut', 'Melon', 'Peanut', 'Weak', 'Honey']:
            if status in abl:
                return {'status': status}
            
    if effect == 'ModifyStats':
        dct = {}
        if '%' in abl:
            if 'Attack' in abl:
                percent = int(re.findall('(\d+)(?=\%)', abl)[0])
                dct['add_atk_percent'] = percent
            if 'Health' in abl:
                percent = int(re.findall('(\d+)(?=\%)', abl)[0])
                dct['add_hp_percent'] = percent
            
        elif '/' in abl:
            atk = int(re.findall('(?<=\+)(\d+)(?=\/)', abl)[0])
            hp = int(re.findall('(?<=\/\+)(\d+)', abl)[0])
            dct['add_atk'] = atk
            dct['add_hp'] = hp
        
        else:
            atk_search = re.search('(?<=\+|ain )(\d+)(?= Attack)', abl)
            if atk_search:
                dct['add_atk'] = int(atk_search.group())
            health_search = re.search('(?<=\+|ain )(\d+)(?= Health)', abl)
            if health_search:
                dct['add_hp'] = int(health_search.group())
        
        if 'until end of battle.' in abl:
            dct['temp'] = True
        return dct
        
    if effect == 'DealDamage':
        dmg_search = re.search('(?<=eal )(\d+)(?= damage)', abl)
        if dmg_search:
            return {'damage': int(dmg_search.group())}
        perc_search = re.search('(\d+)(?=\%)', abl)
        if perc_search:
            return {'damage_perc': int(perc_search.group())}
        
    if effect == 'GainGold':
        gold_search = re.search('(\d+)(?= gold)', abl)
        if gold_search:
            return {'gold': int(gold_search.group())}
        
    if effect == 'DiscountPets':
        gold_search = re.search('(\d+)(?= gold)', abl)
        if gold_search:
            return {'discount': int(gold_search.group())}
        
    if effect == 'GainExperience':
        exp_search = re.search('(\d+)(?= Experience)', abl)
        if exp_search:
            return {'exp': int(exp_search.group())}
        
    if effect == 'Summon':
        dct = {}
        pet_search = re.search('Loyal Chinchilla|Cricket|Guinea Pig|Dirty Rat|Ram|Ant|Bus|Chick', abl)
        if pet_search:
            dct['pet'] = pet_search.group()
        else:
            dct['pet'] = ''
        dct['n_pets'] = 1
        
        lvl_search = re.search('(?<=Lvl. |evel )(\d)', abl)
        if lvl_search:
            dct['lvl'] = int(lvl_search.group())
        else:
            dct['lvl'] = lvl
            
        return dct
        
    return effect_info


df4.Effect_info = df4.apply(lambda x: get_Effect_info(x.Ability, x.Effect, x.name[1], x.Effect_info), axis=1)
# df4.to_excel('temp.xlsx')

In [202]:
df5.Trigger.str.title()

Name         Level
Ant          1                  Faint
             2                  Faint
             3                  Faint
Beaver       1                   Sell
             2                   Sell
                           ...       
Walrus       2                  Faint
             3                  Faint
White Tiger  1        Start Of Battle
             2        Start Of Battle
             3        Start Of Battle
Name: Trigger, Length: 528, dtype: object

In [209]:
df5 = pd.read_excel('pets_df.xlsx', index_col=[0,1])
def get_bywhom(trig):
    if 'riend ahead' in trig:
        return 'FriendAhead'
    if 'nemy' in trig:
        return 'Enemy'
    if 'riend' in trig:
        return 'Friend'
    if 'Pet' in trig:
        return 'TeamPet'
    if trig == 'NoTrigger' or 'turn' in trig or 'battle' in trig:
        return 'No Actor'
    else:
        return 'Self'

def get_trigger(trig, bywhom):
    trig = trig.replace('Friend ahead', '').replace('Friendly pet ', '').replace('Friend', '').replace('Enemy', '').strip()
    if trig == 'NoTrigger':
        trig = 'No Trigger'
    return trig.title().replace(' ', '').replace('-', '')
    
df5.Trigger = df5.apply(lambda x: get_trigger(x.Trigger, x.Trigger_bywhom), axis=1)
# df5.to_excel('temp.xlsx')


In [219]:
df6 = pd.read_excel('pets_df.xlsx', index_col = [0,1])
df6.Attack = df6.Attack * df6.index.get_level_values(1)
df6.Health = df6.Health * df6.index.get_level_values(1)

# df6.to_excel('temp.xlsx')

# Food Dataframe

In [17]:
# columns to add: Is_buff, Is_status, Is_targeted, Atk_buff, Hp_buff, Is_temporary, Status

In [2]:
with io.open(os.path.join(sys.path[0], "18.0_food.json"), mode="r", encoding="utf-8") as f:
    data = json.load(f)
    
fd = pd.DataFrame(data)

col_dict = {}
col_dict['Is_status'] = fd.Ability.str.replace('Give a pet ', '').str.rstrip('.').str.isalpha()
col_dict['Is_status'][13] = True  # meat bone
col_dict['Is_status'][22] = True  # fortune cookie

col_dict['Is_buff'] =  ~col_dict['Is_status'] # lollipop, canned food, chocolate, sleeping pill should be false

col_dict['Is_temporary'] = fd.Ability.str.find('until end of battle') >= 0
col_dict['Is_targeted'] = fd.Ability.str.find('random') < 0
    
fd1 = pd.concat([fd, pd.DataFrame(col_dict)], axis = 1)
fd1.loc[fd1.Name == 'Lollipop', 'Is_buff'] = False
fd1.loc[fd1.Name == 'Canned Food', 'Is_buff'] = False
fd1.loc[fd1.Name == 'Chocolate', 'Is_buff'] = False
fd1.loc[fd1.Name == 'Sleeping Pill', 'Is_buff'] = False

def get_atk(is_buff, abl):
    if not is_buff:
        return np.nan
    
    index = abl.find('/')
    if index >= 0:
        return int(abl[index-1])
    index2 = abl.find(' Attack')
    if index2 >= 0:
        return int(abl[index2-1])
    return 0

def get_hp(is_buff, abl):
    if not is_buff:
        return np.nan
    
    index = abl.find('/')
    if index >= 0:
        return int(abl[index+2])
    index2 = abl.find(' Health')
    if index2 >= 0:
        return int(abl[index2-1])
    return 0
    
fd1['Atk_buff'] = fd1.apply(lambda x: get_atk(x.Is_buff, x.Ability), axis=1)
fd1['Hp_buff'] = fd1.apply(lambda x: get_hp(x.Is_buff, x.Ability), axis=1)

fd1.head()

,Ability,Id,Name,Tier,Packs,Rollable,Is_status,Is_buff,Is_temporary,Is_targeted,Atk_buff,Hp_buff
0,Give a pet +1/+1.,0,Apple,1,"[Pack1, Pack2]",True,False,True,False,True,1.0,1.0
1,Give a pet +2 Attack.,2,Bacon,1,[Custom],True,False,True,False,True,2.0,0.0
2,Give a pet +3/+6.,103,Best Milk,1,[],False,False,True,False,True,3.0,6.0
3,Give a pet +2/+4.,102,Better Milk,1,[],False,False,True,False,True,2.0,4.0
4,Give a pet +2/+2 until end of battle.,27,Cookie,1,[Custom],True,False,True,True,True,2.0,2.0


In [4]:
bool_dict = {}
for item in ['Pack1', 'Pack2', 'Pack3', 'Custom']:
    bool_dict[item] = fd1.Packs.apply(lambda x: item in x)
packs = pd.DataFrame(bool_dict)

fd2 = pd.concat([fd1.drop('Packs', axis = 1), packs], axis = 1)
fd2.to_excel('foods_df.xlsx')

In [150]:
pets_df.columns[5:15]

Index(['Trigger', 'Ability', 'Target', 'N_targets', 'Target_pos', 'Effect',
       'Effect_from', 'Effect_info', 'Effect_repeat', 'N_uses'],
      dtype='object')

In [173]:
fd2[fd2.Is_status].Name.append(pd.Series(['Poison']))

5              Honey
8         Strawberry
10         Croissant
13         Meat Bone
15          Cucumber
16            Garlic
18         Pineapple
21            Cheese
22    Fortune Cookie
23            Grapes
25            Carrot
26             Chili
28             Lemon
29            Pepper
35             Melon
36          Mushroom
39          Popcorns
41             Steak
0             Poison
dtype: object

In [179]:
status_df = pd.DataFrame(index = fd2[fd2.Is_status].Name.append(pd.Series(['Poison', 'Weak'])), columns = pets_df.columns[5:15])
status_df.Trigger = 'Faint, NoTrigger, End turn, Clash, End turn, Clash, NoTrigger, Clash, Clash, Start of turn, \
    End turn, Clash, Clash, Clash, Clash, Faint, Faint, Clash, Clash, Clash'.split(', ')
# status_df.Effect = 'Summon, NoEffect, ModifyStats, '
status_df.Effect_repeat = 1
status_df.N_uses = 999
status_df.loc[['Honey', 'Meat Bone', 'Cheese', 'Pepper', 'Melon', 'Mushroom', 'Popcorns', 'Steak'], 'N_uses'] = 1

status_df

,Trigger,Ability,Target,N_targets,Target_pos,Effect,Effect_from,Effect_info,Effect_repeat,N_uses
Honey,Faint,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
Strawberry,NoTrigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,999
Croissant,End turn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,999
Meat Bone,Clash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
Cucumber,End turn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,999
Garlic,Clash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,999
Pineapple,NoTrigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,999
Cheese,Clash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
Fortune Cookie,Clash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,999
Grapes,Start of turn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,999
